In [1]:
import numpy as np
from typing import List, Tuple

class MOGA:
    """
    Multi-Objective Genetic Algorithm (MOGA) implementation.
    This class implements a MOGA for solving multi-objective optimization problems.
    """

    def __init__(self, pop_size: int, num_objectives: int, gene_size: int, 
                 crossover_rate: float, mutation_rate: float):
        """
        Initialize the MOGA.

        Args:
            pop_size (int): Size of the population.
            num_objectives (int): Number of objectives to optimize.
            gene_size (int): Number of genes in each individual.
            crossover_rate (float): Probability of crossover occurring.
            mutation_rate (float): Probability of mutation for each gene.
        """
        self.pop_size = pop_size
        self.num_objectives = num_objectives
        self.gene_size = gene_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = self.initialize_population()

    def initialize_population(self) -> np.ndarray:
        """
        Initialize a random population.

        Returns:
            np.ndarray: A 2D array where each row represents an individual.
        """
        return np.random.rand(self.pop_size, self.gene_size)

    def evaluate_fitness(self, individual: np.ndarray) -> List[float]:
        """
        Evaluate the fitness of an individual.
        This is a placeholder and should be replaced with actual objective functions.

        Args:
            individual (np.ndarray): The individual to evaluate.

        Returns:
            List[float]: A list of fitness values, one for each objective.
        """
        return [np.sum(individual) for _ in range(self.num_objectives)]

    def fast_non_dominated_sort(self, fitness_values: np.ndarray) -> List[List[int]]:
        """
        Perform fast non-dominated sorting to identify Pareto fronts.

        Args:
            fitness_values (np.ndarray): Fitness values for the entire population.

        Returns:
            List[List[int]]: A list of Pareto fronts, where each front is a list of indices.
        """
        fronts = [[]]
        dominated_solutions = [[] for _ in range(self.pop_size)]
        domination_counts = [0] * self.pop_size
        
        for i in range(self.pop_size):
            for j in range(i+1, self.pop_size):
                if np.all(fitness_values[i] <= fitness_values[j]) and np.any(fitness_values[i] < fitness_values[j]):
                    dominated_solutions[i].append(j)
                    domination_counts[j] += 1
                elif np.all(fitness_values[j] <= fitness_values[i]) and np.any(fitness_values[j] < fitness_values[i]):
                    dominated_solutions[j].append(i)
                    domination_counts[i] += 1
            
            if domination_counts[i] == 0:
                fronts[0].append(i)
        
        k = 0
        while fronts[k]:
            next_front = []
            for i in fronts[k]:
                for j in dominated_solutions[i]:
                    domination_counts[j] -= 1
                    if domination_counts[j] == 0:
                        next_front.append(j)
            k += 1
            fronts.append(next_front)
        
        return fronts[:-1]  # Remove the last empty front

    def crowding_distance(self, fitness_values: np.ndarray, front: List[int]) -> np.ndarray:
        """
        Calculate crowding distance for individuals in a front.

        Args:
            fitness_values (np.ndarray): Fitness values for the entire population.
            front (List[int]): Indices of individuals in the current front.

        Returns:
            np.ndarray: Crowding distances for individuals in the front.
        """
        distances = np.zeros(len(front))
        for obj in range(self.num_objectives):
            sorted_front = sorted(front, key=lambda x: fitness_values[x, obj])
            distances[0] = distances[-1] = np.inf
            for i in range(1, len(front) - 1):
                distances[i] += (fitness_values[sorted_front[i+1], obj] - 
                                 fitness_values[sorted_front[i-1], obj])
        return distances

    def select_parents(self, population: np.ndarray, fitness_values: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Select two parents for reproduction using tournament selection based on Pareto dominance and crowding distance.

        Args:
            population (np.ndarray): The current population.
            fitness_values (np.ndarray): Fitness values for the entire population.

        Returns:
            Tuple[np.ndarray, np.ndarray]: Two selected parent individuals.
        """
        fronts = self.fast_non_dominated_sort(fitness_values)
        selected = []
        
        for front in fronts:
            if len(selected) + len(front) <= self.pop_size:
                selected.extend(front)
            else:
                distances = self.crowding_distance(fitness_values, front)
                sorted_front = sorted(zip(front, distances), key=lambda x: -x[1])
                selected.extend([x[0] for x in sorted_front[:self.pop_size - len(selected)]])
                break
        
        idx1, idx2 = np.random.choice(selected, 2, replace=False)
        return population[idx1], population[idx2]

    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:
        """
        Perform crossover between two parents.

        Args:
            parent1 (np.ndarray): First parent.
            parent2 (np.ndarray): Second parent.

        Returns:
            np.ndarray: Child produced by crossover.
        """
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.gene_size)
            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        else:
            child = parent1.copy()
        return child

    def mutate(self, individual: np.ndarray) -> np.ndarray:
        """
        Perform mutation on an individual.

        Args:
            individual (np.ndarray): The individual to mutate.

        Returns:
            np.ndarray: The mutated individual.
        """
        mutation_mask = np.random.rand(self.gene_size) < self.mutation_rate
        individual[mutation_mask] = np.random.rand(np.sum(mutation_mask))
        return individual

    def run(self, generations: int) -> List[np.ndarray]:
        """
        Run the MOGA for a specified number of generations.

        Args:
            generations (int): Number of generations to run.

        Returns:
            List[np.ndarray]: The final Pareto front.
        """
        for _ in range(generations):
            fitness_values = np.array([self.evaluate_fitness(ind) for ind in self.population])
            new_population = []
            
            while len(new_population) < self.pop_size:
                parent1, parent2 = self.select_parents(self.population, fitness_values)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                new_population.append(child)
            
            self.population = np.array(new_population)
        
        return self.get_pareto_front()

    def get_pareto_front(self) -> List[np.ndarray]:
        """
        Get the Pareto front from the current population.

        Returns:
            List[np.ndarray]: The Pareto front (non-dominated solutions).
        """
        fitness_values = np.array([self.evaluate_fitness(ind) for ind in self.population])
        fronts = self.fast_non_dominated_sort(fitness_values)
        return [self.population[i] for i in fronts[0]]

# Usage example
moga = MOGA(pop_size=100, num_objectives=2, gene_size=10, crossover_rate=0.8, mutation_rate=0.1)
pareto_front = moga.run(generations=50)
